In [ ]:
!pip uninstall -y jax jaxlib
!pip install --upgrade --quiet jax jaxlib


Found existing installation: jax 0.6.2
Uninstalling jax-0.6.2:
  Successfully uninstalled jax-0.6.2
Found existing installation: jaxlib 0.6.2
Uninstalling jaxlib-0.6.2:
  Successfully uninstalled jaxlib-0.6.2


In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm


/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.6.2, so it will not be used.
  warnings.warn(


In [ ]:
# --------- Settings ---------
img_size = 299
base_path = "/content/drive/MyDrive/graduation project/trial_split"
splits = ["train", "val", "test"]
classes = ['AMD', 'DME', 'ERM', 'NO']
model_path = "/content/drive/MyDrive/graduation project/models/OCT_InceptionV3.h5"
output_path = "/content/drive/MyDrive/graduation project/features"

# --------- Load Trained Model and Create Feature Extractor ---------
full_model = load_model(model_path)
feature_extractor = Model(inputs=full_model.input, outputs=full_model.layers[-3].output)

In [ ]:
# --------- Feature Extraction Function ---------
def extract_features(split):
    features = []
    labels = []
    for class_index, class_name in enumerate(classes):
        class_path = os.path.join(base_path, split, class_name)
        image_files = os.listdir(class_path)
        for img_file in tqdm(image_files, desc=f"{split}/{class_name}"):
            img_path = os.path.join(class_path, img_file)
            try:
                img = image.load_img(img_path, target_size=(img_size, img_size))
                img_array = image.img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)
                img_array = preprocess_input(img_array)
                feature = feature_extractor.predict(img_array, verbose=0)
                features.append(feature.flatten())
                labels.append(class_index)
            except Exception as e:
                print(f"⚠️ Error with image {img_path}: {e}")
    return np.array(features), np.array(labels)


In [ ]:
# --------- Extract and Save Features ---------
os.makedirs(output_path, exist_ok=True)
for split in splits:
    X, y = extract_features(split)
    np.save(os.path.join(output_path, f"X_{split}.npy"), X)
    np.save(os.path.join(output_path, f"y_{split}.npy"), y)

test/NO: 100%|██████████| 200/200 [01:28<00:00,  2.25it/s]


In [ ]:
# --------- Load Saved Features ---------
X_train = np.load(os.path.join(output_path, "X_train.npy"))
y_train = np.load(os.path.join(output_path, "y_train.npy"))
X_val = np.load(os.path.join(output_path, "X_val.npy"))
y_val = np.load(os.path.join(output_path, "y_val.npy"))
X_test = np.load(os.path.join(output_path, "X_test.npy"))
y_test = np.load(os.path.join(output_path, "y_test.npy"))


In [ ]:
# --------- Train SVM Classifier ---------
print("\n🔧 Training SVM...")
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train, y_train)
pred_svm = svm.predict(X_test)
print("\n📊 SVM Results:")
print(classification_report(y_test, pred_svm, target_names=classes))
print("Accuracy:", accuracy_score(y_test, pred_svm))



🔧 Training SVM...

📊 SVM Results:
              precision    recall  f1-score   support

         AMD       0.94      0.96      0.95       186
         DME       0.95      0.95      0.95       177
         ERM       0.95      0.94      0.94       186
          NO       0.96      0.96      0.96       200

    accuracy                           0.95       749
   macro avg       0.95      0.95      0.95       749
weighted avg       0.95      0.95      0.95       749

Accuracy: 0.951935914552737


In [ ]:

# --------- Train XGBoost (GradientBoosting) ---------
print("\n🔧 Training XGBoost...")
xgb = GradientBoostingClassifier()
xgb.fit(X_train, y_train)
pred_xgb = xgb.predict(X_test)
print("\n📊 XGBoost Results:")
print(classification_report(y_test, pred_xgb, target_names=classes))
print("Accuracy:", accuracy_score(y_test, pred_xgb))



🔧 Training XGBoost...

📊 XGBoost Results:
              precision    recall  f1-score   support

         AMD       0.95      0.95      0.95       186
         DME       0.93      0.95      0.94       177
         ERM       0.95      0.93      0.94       186
          NO       0.96      0.96      0.96       200

    accuracy                           0.95       749
   macro avg       0.95      0.95      0.95       749
weighted avg       0.95      0.95      0.95       749

Accuracy: 0.9492656875834445


In [ ]:
# 🔁 Ensemble (SVM + XGBoost) Classifier on Extracted Features

import numpy as np
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# --------- Load Features ---------
X_train = np.load("/content/drive/MyDrive/graduation project/features/X_train.npy")
y_train = np.load("/content/drive/MyDrive/graduation project/features/y_train.npy")
X_test = np.load("/content/drive/MyDrive/graduation project/features/X_test.npy")
y_test = np.load("/content/drive/MyDrive/graduation project/features/y_test.npy")

# --------- Normalize Features ---------
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# --------- Define Classifiers ---------
svm_clf = SVC(kernel='rbf', probability=True, C=10, gamma=0.01, random_state=42)
xgb_clf = XGBClassifier(n_estimators=300, max_depth=5, learning_rate=0.05, use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# --------- Voting Classifier ---------
ensemble_model = VotingClassifier(estimators=[
    ('svm', svm_clf),
    ('xgb', xgb_clf)
], voting='soft')

# --------- Train Ensemble ---------
print("\n🔧 Training Ensemble (SVM + XGBoost)...")
ensemble_model.fit(X_train, y_train)

# --------- Evaluate ---------
y_pred = ensemble_model.predict(X_test)
print("\n📊 Ensemble Results:")
print(classification_report(y_test, y_pred, target_names=['AMD', 'DME', 'ERM', 'NO']))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")



🔧 Training Ensemble (SVM + XGBoost)...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:09:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



📊 Ensemble Results:
              precision    recall  f1-score   support

         AMD       0.95      0.95      0.95       186
         DME       0.94      0.95      0.95       177
         ERM       0.95      0.94      0.94       186
          NO       0.97      0.97      0.97       200

    accuracy                           0.95       749
   macro avg       0.95      0.95      0.95       749
weighted avg       0.95      0.95      0.95       749

Accuracy: 0.9532710280373832


In [ ]:
import os
import numpy as np
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tqdm import tqdm

# ---------- Settings ----------
img_size = 299
base_path = "/content/drive/MyDrive/graduation project/trial_split"
splits = ["train", "val", "test"]
classes = ['AMD', 'DME', 'ERM', 'NO']
model_path = "/content/drive/MyDrive/graduation project/models/OCT_InceptionV3.h5"

# ---------- Load Model and Build Feature Extractor ----------
full_model = load_model(model_path)
feature_extractor = Model(inputs=full_model.input, outputs=full_model.layers[-3].output)  # Dropout output

# ---------- Extract Features Function ----------
def extract_features(split):
    features, labels = [], []
    for class_index, class_name in enumerate(classes):
        class_dir = os.path.join(base_path, split, class_name)
        for img_file in tqdm(os.listdir(class_dir), desc=f"{split}/{class_name}"):
            img_path = os.path.join(class_dir, img_file)
            try:
                img = image.load_img(img_path, target_size=(img_size, img_size))
                img_array = image.img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)
                img_array = preprocess_input(img_array)
                feature = feature_extractor.predict(img_array, verbose=0)
                features.append(feature.flatten())
                labels.append(class_index)
            except Exception as e:
                print(f"⚠️ Error with {img_path}: {e}")
    return np.array(features), np.array(labels)

# ---------- Extract All Features ----------
print("🔍 Extracting train features...")
X_train, y_train = extract_features("train")
print("🔍 Extracting val features...")
X_val, y_val = extract_features("val")
print("🔍 Extracting test features...")
X_test, y_test = extract_features("test")

# ---------- Merge Train + Val ----------
X_combined = np.concatenate([X_train, X_val], axis=0)
y_combined = np.concatenate([y_train, y_val], axis=0)

# ---------- Normalize Features ----------
scaler = StandardScaler()
X_combined = scaler.fit_transform(X_combined)
X_test = scaler.transform(X_test)

# ---------- Train and Evaluate Each Model Separately ----------

# 🧠 SVM
print("\n🔧 Training SVM...")
svm_model = SVC(kernel='rbf', C=10, gamma='scale', probability=True)
svm_model.fit(X_combined, y_combined)
svm_preds = svm_model.predict(X_test)
print("\n📊 SVM Results:")
print(classification_report(y_test, svm_preds, target_names=classes))
print(f"Accuracy: {accuracy_score(y_test, svm_preds):.4f}")

# 🌲 Random Forest
print("\n🔧 Training Random Forest...")
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_combined, y_combined)
rf_preds = rf_model.predict(X_test)
print("\n📊 Random Forest Results:")
print(classification_report(y_test, rf_preds, target_names=classes))
print(f"Accuracy: {accuracy_score(y_test, rf_preds):.4f}")

# ⚡ XGBoost
print("\n🔧 Training XGBoost...")
xgb_model = XGBClassifier(n_estimators=200, learning_rate=0.05, use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_combined, y_combined)
xgb_preds = xgb_model.predict(X_test)
print("\n📊 XGBoost Results:")
print(classification_report(y_test, xgb_preds, target_names=classes))
print(f"Accuracy: {accuracy_score(y_test, xgb_preds):.4f}")


🔍 Extracting train features...


train/NO: 100%|██████████| 929/929 [06:44<00:00,  2.30it/s]


🔍 Extracting val features...


val/NO: 100%|██████████| 199/199 [01:27<00:00,  2.27it/s]


🔍 Extracting test features...


test/NO: 100%|██████████| 200/200 [01:26<00:00,  2.30it/s]



🔧 Training SVM...

📊 SVM Results:
              precision    recall  f1-score   support

         AMD       0.95      0.95      0.95       186
         DME       0.95      0.95      0.95       177
         ERM       0.95      0.92      0.94       186
          NO       0.96      0.97      0.97       200

    accuracy                           0.95       749
   macro avg       0.95      0.95      0.95       749
weighted avg       0.95      0.95      0.95       749

Accuracy: 0.9506

🔧 Training Random Forest...

📊 Random Forest Results:
              precision    recall  f1-score   support

         AMD       0.95      0.96      0.95       186
         DME       0.95      0.95      0.95       177
         ERM       0.95      0.93      0.94       186
          NO       0.97      0.97      0.97       200

    accuracy                           0.95       749
   macro avg       0.95      0.95      0.95       749
weighted avg       0.95      0.95      0.95       749

Accuracy: 0.9533

🔧 Tra

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:51:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



📊 XGBoost Results:
              precision    recall  f1-score   support

         AMD       0.95      0.95      0.95       186
         DME       0.94      0.94      0.94       177
         ERM       0.94      0.93      0.94       186
          NO       0.96      0.97      0.97       200

    accuracy                           0.95       749
   macro avg       0.95      0.95      0.95       749
weighted avg       0.95      0.95      0.95       749

Accuracy: 0.9493


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

# -------- Settings --------
img_size = 224
base_path = "/content/drive/MyDrive/graduation project/trial_split"
splits = ["train", "val", "test"]
classes = ['AMD', 'DME', 'ERM', 'NO']
model_weights_path = "/content/drive/MyDrive/graduation project/models/squeezNet.pth"

# -------- Dataset Loader --------
class OCTDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx]

# -------- Load Images --------
def load_images_and_labels(split):
    images, labels = [], []
    for label_index, class_name in enumerate(classes):
        class_path = os.path.join(base_path, split, class_name)
        for img_file in os.listdir(class_path):
            images.append(os.path.join(class_path, img_file))
            labels.append(label_index)
    return images, labels

# -------- Transform --------
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])



# -------- Prepare Data --------
train_imgs, train_labels = load_images_and_labels("train")
val_imgs, val_labels = load_images_and_labels("val")
test_imgs, test_labels = load_images_and_labels("test")

X_train = extract_features(train_imgs)
X_val = extract_features(val_imgs)
X_test = extract_features(test_imgs)
y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)

# -------- Train & Evaluate ML Models --------
def evaluate_model(model, X_train, y_train, X_test, y_test, name):
    print(f"\n🔧 Training {name}...")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(f"\n📊 {name} Results:")
    print(classification_report(y_test, preds, target_names=classes))
    print(f"Accuracy: {acc:.4f}")

# -------- SVM --------
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True)
evaluate_model(svm, X_train, y_train, X_test, y_test, "SVM")

# -------- Random Forest --------
rf = RandomForestClassifier(n_estimators=200, max_depth=20, random_state=42)
evaluate_model(rf, X_train, y_train, X_test, y_test, "Random Forest")

# -------- XGBoost --------
xgb = XGBClassifier(n_estimators=200, learning_rate=0.1, use_label_encoder=False, eval_metric='mlogloss')
evaluate_model(xgb, X_train, y_train, X_test, y_test, "XGBoost")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Extracting features: 100%|██████████| 749/749 [00:56<00:00, 13.25it/s]



🔧 Training SVM...

📊 SVM Results:
              precision    recall  f1-score   support

         AMD       0.95      0.95      0.95       186
         DME       0.95      0.94      0.95       177
         ERM       0.96      0.96      0.96       186
          NO       0.96      0.98      0.97       200

    accuracy                           0.96       749
   macro avg       0.96      0.96      0.96       749
weighted avg       0.96      0.96      0.96       749

Accuracy: 0.9573

🔧 Training Random Forest...

📊 Random Forest Results:
              precision    recall  f1-score   support

         AMD       0.91      0.98      0.95       186
         DME       0.97      0.93      0.95       177
         ERM       0.95      0.96      0.95       186
          NO       0.98      0.94      0.96       200

    accuracy                           0.95       749
   macro avg       0.95      0.95      0.95       749
weighted avg       0.95      0.95      0.95       749

Accuracy: 0.9519

🔧 Tra

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [04:00:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
!pip install scikit-learn xgboost


In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tqdm import tqdm

# --------- Settings ---------
img_size = 224
base_path = "/content/drive/MyDrive/graduation project/trial_split"
splits = ["train", "val", "test"]
classes = ['AMD', 'DME', 'ERM', 'NO']
model_path = "/content/drive/MyDrive/graduation project/models/model0.h5"
output_path = "/content/drive/MyDrive/graduation project/features_vgg16"

os.makedirs(output_path, exist_ok=True)

# --------- Load Your VGG16 Model and Remove Last Layer ---------
full_model = load_model(model_path)
feature_extractor = Model(inputs=full_model.input, outputs=full_model.layers[-3].output)  # Use the Flatten layer

# --------- Function to Extract Features ---------
def extract_features(split):
    features = []
    labels = []

    for class_index, class_name in enumerate(classes):
        class_path = os.path.join(base_path, split, class_name)
        image_files = os.listdir(class_path)

        for img_file in tqdm(image_files, desc=f"{split}/{class_name}"):
            img_path = os.path.join(class_path, img_file)
            try:
                img = image.load_img(img_path, target_size=(img_size, img_size))
                img_array = image.img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)
                img_array = preprocess_input(img_array)

                feature = feature_extractor.predict(img_array, verbose=0)
                features.append(feature.flatten())
                labels.append(class_index)
            except Exception as e:
                print(f"⚠️ Error with image {img_path}: {e}")

    return np.array(features), np.array(labels)

# --------- Extract & Save ---------
for split in splits:
    print(f"\n🔍 Extracting features for {split}...")
    X, y = extract_features(split)
    np.save(os.path.join(output_path, f"X_{split}.npy"), X)
    np.save(os.path.join(output_path, f"y_{split}.npy"), y)
    print(f"✅ Done: {split} → Features: {X.shape}, Labels: {y.shape}")



🔍 Extracting features for train...


train/NO: 100%|██████████| 929/929 [12:27<00:00,  1.24it/s]


✅ Done: train → Features: (3481, 512), Labels: (3481,)

🔍 Extracting features for val...


val/NO: 100%|██████████| 199/199 [02:41<00:00,  1.23it/s]


✅ Done: val → Features: (745, 512), Labels: (745,)

🔍 Extracting features for test...


test/NO: 100%|██████████| 200/200 [02:42<00:00,  1.23it/s]

✅ Done: test → Features: (749, 512), Labels: (749,)


In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load features
X_train = np.load(output_path + "/X_train.npy")
y_train = np.load(output_path + "/y_train.npy")
X_val = np.load(output_path + "/X_val.npy")
y_val = np.load(output_path + "/y_val.npy")
X_test = np.load(output_path + "/X_test.npy")
y_test = np.load(output_path + "/y_test.npy")

# Combine train + val
X_train_full = np.concatenate([X_train, X_val], axis=0)
y_train_full = np.concatenate([y_train, y_val], axis=0)


In [ ]:
print("\n🔧 Training SVM...")
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True)
svm.fit(X_train_full, y_train_full)
y_pred = svm.predict(X_test)
print("\n📊 SVM Results:")
print(classification_report(y_test, y_pred, target_names=classes))
print("Accuracy:", accuracy_score(y_test, y_pred))



🔧 Training SVM...

📊 SVM Results:
              precision    recall  f1-score   support

         AMD       0.93      0.89      0.91       186
         DME       0.89      0.90      0.90       177
         ERM       0.89      0.89      0.89       186
          NO       0.91      0.94      0.92       200

    accuracy                           0.90       749
   macro avg       0.90      0.90      0.90       749
weighted avg       0.90      0.90      0.90       749

Accuracy: 0.9038718291054739


In [ ]:
print("\n🔧 Training Random Forest...")
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train_full, y_train_full)
y_pred = rf.predict(X_test)
print("\n📊 Random Forest Results:")
print(classification_report(y_test, y_pred, target_names=classes))
print("Accuracy:", accuracy_score(y_test, y_pred))



🔧 Training Random Forest...

📊 Random Forest Results:
              precision    recall  f1-score   support

         AMD       0.92      0.89      0.90       186
         DME       0.84      0.81      0.82       177
         ERM       0.80      0.80      0.80       186
          NO       0.82      0.87      0.84       200

    accuracy                           0.84       749
   macro avg       0.84      0.84      0.84       749
weighted avg       0.84      0.84      0.84       749

Accuracy: 0.8411214953271028


In [ ]:
print("\n🔧 Training XGBoost...")
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb.fit(X_train_full, y_train_full)
y_pred = xgb.predict(X_test)
print("\n📊 XGBoost Results:")
print(classification_report(y_test, y_pred, target_names=classes))
print("Accuracy:", accuracy_score(y_test, y_pred))



🔧 Training XGBoost...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:44:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



📊 XGBoost Results:
              precision    recall  f1-score   support

         AMD       0.94      0.90      0.92       186
         DME       0.90      0.88      0.89       177
         ERM       0.87      0.84      0.86       186
          NO       0.84      0.92      0.88       200

    accuracy                           0.89       749
   macro avg       0.89      0.89      0.89       749
weighted avg       0.89      0.89      0.89       749

Accuracy: 0.8865153538050734
